In [179]:
# Import the necessary libraries
import pandas as pd
import os
import requests
import time
import numpy as np

In [161]:
# Create a dataframe where we store the game titles
igdb_games_df = pd.read_csv("games.csv", skipinitialspace=True, usecols=['id', 'name', 'first_release_date', 'involved_companies', 'game_modes', 'genres'])
print("Initial size: ", len(igdb_games_df))
igdb_games_df.drop(igdb_games_df[igdb_games_df.first_release_date < 1443564000].index, inplace=True)
print("Final size: ", len(igdb_games_df))
igdb_games_df.head()

Initial size:  6349
Final size:  1975


,id,first_release_date,game_modes,genres,involved_companies,name
37,117,1480982400,"[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam...","[{'id': 36389, 'company': {'id': 44, 'name': '...",The Last Guardian
90,359,1480377600,"[{'id': 1, 'name': 'Single player'}, {'id': 3,...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...","[{'id': 53470, 'company': {'id': 2670, 'name':...",Final Fantasy XV
400,1338,1444089600,"[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 13, 'name': 'Simulator'}, {'id': 15, '...","[{'id': 201720, 'company': {'id': 1224, 'name'...",Prison Architect
668,2112,1465257600,"[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 4, 'name': 'Fighting'}, {'id': 31, 'na...","[{'id': 7372, 'company': {'id': 1, 'name': 'El...",Mirror's Edge Catalyst
669,2113,1447718400,"[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 5, 'name': 'Shooter'}]","[{'id': 56415, 'company': {'id': 1, 'name': 'E...",Star Wars Battlefront


In [162]:
# URL and credentials for Opencritic API
url = "https://opencritic-api.p.rapidapi.com/game/search"

headers = {
    'X-RapidAPI-Key':os.getenv("X-RAPIDAPI-KEY"),
    'X-RapidAPI-Host':os.getenv("X-RAPIDAPI-HOST")
}

In [170]:
# Define schema for the dataframe where we store the id, distance, and the name from Opencritic
games_info_df = pd.DataFrame(columns = ['igdb_name', 'opencritic_name', 'opencritic_id', 'igdb_id', 'dist', 'first_release_date', 'involved_companies', 'game_modes', 'genres'])

In [172]:
# For each game, extract the information from Opencritic and combine them with the IGDB information
for index, igdb_game in igdb_games_df.iterrows():
    response = requests.get(url, headers=headers, params={"criteria":igdb_game['name']}).json()
    min_distance_dict = min(response, key=lambda x: x['dist'])
    # Create a DataFrame with the combined information of opencritic and igdb
    games_info_df.loc[len(games_info_df)] = {'igdb_name': igdb_game['name'], 
                                            'opencritic_name': min_distance_dict['name'],
                                            'opencritic_id': min_distance_dict['id'],
                                            'igdb_id': igdb_game['id'],
                                            'dist': min_distance_dict['dist'],
                                            'first_release_date': igdb_game['first_release_date'],
                                            'involved_companies': igdb_game['involved_companies'],
                                            'game_modes': igdb_game['game_modes'],
                                            'genres': igdb_game['genres']}
    
    # Wait for some time to not reach the 100 requests per second
    time.sleep(0.02)

games_info_df.head()

,igdb_name,opencritic_name,opencritic_id,igdb_id,dist,first_release_date,involved_companies,game_modes,genres
0,The Last Guardian,The Last Guardian,1524,117,0.0,1480982400,"[{'id': 36389, 'company': {'id': 44, 'name': '...","[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam..."
1,Final Fantasy XV,Final Fantasy XV,1512,359,0.0,1480377600,"[{'id': 53470, 'company': {'id': 2670, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 3,...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i..."
2,Prison Architect,Prison Architect,1942,1338,0.0,1444089600,"[{'id': 201720, 'company': {'id': 1224, 'name'...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 13, 'name': 'Simulator'}, {'id': 15, '..."
3,Mirror's Edge Catalyst,Mirror's Edge Catalyst,1549,2112,0.0,1465257600,"[{'id': 7372, 'company': {'id': 1, 'name': 'El...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 4, 'name': 'Fighting'}, {'id': 31, 'na..."
4,Star Wars Battlefront,Star Wars Battlefront,1511,2113,0.0,1447718400,"[{'id': 56415, 'company': {'id': 1, 'name': 'E...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 5, 'name': 'Shooter'}]"


In [173]:
# Validate that we have extracted correct data
print(games_info_df['first_release_date'].min())
games_info_df.head()

1443571200


,igdb_name,opencritic_name,opencritic_id,igdb_id,dist,first_release_date,involved_companies,game_modes,genres
0,The Last Guardian,The Last Guardian,1524,117,0.0,1480982400,"[{'id': 36389, 'company': {'id': 44, 'name': '...","[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam..."
1,Final Fantasy XV,Final Fantasy XV,1512,359,0.0,1480377600,"[{'id': 53470, 'company': {'id': 2670, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 3,...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i..."
2,Prison Architect,Prison Architect,1942,1338,0.0,1444089600,"[{'id': 201720, 'company': {'id': 1224, 'name'...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 13, 'name': 'Simulator'}, {'id': 15, '..."
3,Mirror's Edge Catalyst,Mirror's Edge Catalyst,1549,2112,0.0,1465257600,"[{'id': 7372, 'company': {'id': 1, 'name': 'El...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 4, 'name': 'Fighting'}, {'id': 31, 'na..."
4,Star Wars Battlefront,Star Wars Battlefront,1511,2113,0.0,1447718400,"[{'id': 56415, 'company': {'id': 1, 'name': 'E...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 5, 'name': 'Shooter'}]"


In [175]:
# Save dataframe as a csv file 
games_info_df.to_csv("opencritic_igdb_info.csv",index=False)

In [252]:
df = pd.read_csv("opencritic_igdb_info.csv")
df

,igdb_name,opencritic_name,opencritic_id,igdb_id,dist,first_release_date,involved_companies,game_modes,genres
0,The Last Guardian,The Last Guardian,1524,117,0.000000,1480982400,"[{'id': 36389, 'company': {'id': 44, 'name': '...","[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam..."
1,Final Fantasy XV,Final Fantasy XV,1512,359,0.000000,1480377600,"[{'id': 53470, 'company': {'id': 2670, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 3,...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i..."
2,Prison Architect,Prison Architect,1942,1338,0.000000,1444089600,"[{'id': 201720, 'company': {'id': 1224, 'name'...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 13, 'name': 'Simulator'}, {'id': 15, '..."
3,Mirror's Edge Catalyst,Mirror's Edge Catalyst,1549,2112,0.000000,1465257600,"[{'id': 7372, 'company': {'id': 1, 'name': 'El...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 4, 'name': 'Fighting'}, {'id': 31, 'na..."
4,Star Wars Battlefront,Star Wars Battlefront,1511,2113,0.000000,1447718400,"[{'id': 56415, 'company': {'id': 1, 'name': 'E...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 5, 'name': 'Shooter'}]"
...,...,...,...,...,...,...,...,...,...
1970,KarmaZoo,Karma Knight,11308,246564,0.687500,1696032000,"[{'id': 211397, 'company': {'id': 5079, 'name'...","[{'id': 2, 'name': 'Multiplayer'}, {'id': 3, '...","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam..."
1971,The Legend of Heroes: Trails from Zero - Delux...,The Legend of Heroes: Trails from Zero,13380,248792,0.294118,1664496000,"[{'id': 213580, 'company': {'id': 2751, 'name'...","[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i..."
1972,The Legend of Heroes: Trails to Azure - Deluxe...,The Legend of Heroes: Trails to Azure,14398,249169,0.300000,1679011200,"[{'id': 214184, 'company': {'id': 35704, 'name...",NaN,"[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i..."
1973,DriftCE,Drifter,1727,249331,0.545455,1683763200,"[{'id': 214364, 'company': {'id': 445, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 10, 'name': 'Racing'}, {'id': 13, 'nam..."


In [285]:
# create the final df, with additional columns to retrieve more useful information
# A bit nasty technique though, it doesn't really follow the best practices, but
# does the trick for now.
final_df = df.copy()
new_columns = ['hasLootBoxes', 'percentRecommended', 
               'numReviews', 'numTopCriticReviews', 
               'medianScore', 'topCriticScore', 
               'tier', 'percentile', 'num_platforms', 
               'embargoDate', 'firstReleaseDate', 'firstReviewDate',
               'latestReviewDate', 'tenthReviewDate', 'criticalReviewDate'
               ]
final_df[new_columns] = np.nan
final_df

,igdb_name,opencritic_name,opencritic_id,igdb_id,dist,first_release_date,involved_companies,game_modes,genres,hasLootBoxes,...,topCriticScore,tier,percentile,num_platforms,embargoDate,firstReleaseDate,firstReviewDate,latestReviewDate,tenthReviewDate,criticalReviewDate
0,The Last Guardian,The Last Guardian,1524,117,0.000000,1480982400,"[{'id': 36389, 'company': {'id': 44, 'name': '...","[{'id': 1, 'name': 'Single player'}]","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Final Fantasy XV,Final Fantasy XV,1512,359,0.000000,1480377600,"[{'id': 53470, 'company': {'id': 2670, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 3,...","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Prison Architect,Prison Architect,1942,1338,0.000000,1444089600,"[{'id': 201720, 'company': {'id': 1224, 'name'...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 13, 'name': 'Simulator'}, {'id': 15, '...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mirror's Edge Catalyst,Mirror's Edge Catalyst,1549,2112,0.000000,1465257600,"[{'id': 7372, 'company': {'id': 1, 'name': 'El...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 4, 'name': 'Fighting'}, {'id': 31, 'na...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Star Wars Battlefront,Star Wars Battlefront,1511,2113,0.000000,1447718400,"[{'id': 56415, 'company': {'id': 1, 'name': 'E...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 5, 'name': 'Shooter'}]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,KarmaZoo,Karma Knight,11308,246564,0.687500,1696032000,"[{'id': 211397, 'company': {'id': 5079, 'name'...","[{'id': 2, 'name': 'Multiplayer'}, {'id': 3, '...","[{'id': 8, 'name': 'Platform'}, {'id': 9, 'nam...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971,The Legend of Heroes: Trails from Zero - Delux...,The Legend of Heroes: Trails from Zero,13380,248792,0.294118,1664496000,"[{'id': 213580, 'company': {'id': 2751, 'name'...","[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972,The Legend of Heroes: Trails to Azure - Deluxe...,The Legend of Heroes: Trails to Azure,14398,249169,0.300000,1679011200,"[{'id': 214184, 'company': {'id': 35704, 'name...",NaN,"[{'id': 12, 'name': 'Role-playing (RPG)'}, {'i...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973,DriftCE,Drifter,1727,249331,0.545455,1683763200,"[{'id': 214364, 'company': {'id': 445, 'name':...","[{'id': 1, 'name': 'Single player'}, {'id': 2,...","[{'id': 10, 'name': 'Racing'}, {'id': 13, 'nam...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [286]:
# Add the new information in the final dataframe
for index, game in final_df.iterrows():
    # retrieve the information from the url of each specific game, based on its id
    response = requests.get("https://opencritic-api.p.rapidapi.com/game/"+str(game['opencritic_id']), headers=headers).json()
    
    # iterate throught eas new column that we added later, and replace the NaN value with the actual value from the API
    for i in range(len(new_columns)): final_df.at[index, new_columns[i]] = response[new_columns[i]] if new_columns[i] in response else np.nan
    
    # Wait for some time to not reach the 100 requests per second
    time.sleep(0.02)

In [288]:
final_df.to_csv("final_df.csv")